In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

# 한글 폰트 문제
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
from xgboost import XGBClassifier

#### DataFrame

In [17]:
df = pd.read_csv('./dataset/gmm_clustering.csv', encoding='utf-8')
df = df[['cluster', 'avg_score', 'monthly_loan', 'monthly_bk_loan', 'monthly_cd_loan', 'monthly_installments_loan', 'monthly_insurance_loan', 'monthly_sbk_loan', 'credit_loan', 'mortgage_loan', 'inst_rep_loanb', 'ls_rep_loanb']]
df.head()

,cluster,avg_score,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,credit_loan,mortgage_loan,inst_rep_loanb,ls_rep_loanb
0,2,740,24940000,18500000,620000,2110000,680000,1320000,14920000,35450000,10520000,9750000
1,2,782,30880000,24150000,680000,2300000,790000,1160000,19070000,42420000,12220000,12240000
2,2,782,31000000,24240000,680000,2300000,790000,1210000,19190000,42480000,12280000,12370000
3,2,782,31390000,24570000,680000,2300000,790000,1230000,19360000,42980000,12320000,12700000
4,2,781,31730000,24830000,700000,2320000,770000,1250000,19580000,43370000,12420000,12870000


In [18]:
# 대출 상품 별 금액 비중
list = []
for i in range(7):
    tmp = []
    dataframe = df[df['cluster']==i]
    dataframe.drop(['cluster', 'avg_score', 'monthly_loan', 'inst_rep_loanb', 'ls_rep_loanb'], axis=1, inplace=True)
    total = sum(round(dataframe.mean(), 3))
    tmp.append(total)
    for col in dataframe.columns:
        ratio = round((dataframe[col].mean()/total)*100, 2)
        tmp.append(ratio)
    list.append(tmp)
loan = pd.DataFrame(list, columns=['total_loan', '은행', '카드', '캐피탈', '보험', '저축은행', '신용', '담보'])
loan

,total_loan,은행,카드,캐피탈,보험,저축은행,신용,담보
0,175156277.53400,34.76000,0.17000,0.35000,0.89000,0.13000,9.37000,54.34000
1,163625244.12300,27.71000,0.81000,2.90000,1.72000,0.71000,16.86000,49.29000
2,80698379.62900,23.13000,0.87000,4.74000,1.10000,2.00000,20.27000,47.88000
3,201372848.66600,29.26000,0.56000,1.96000,1.13000,0.34000,14.66000,52.10000
4,117375599.99900,32.87000,0.12000,0.42000,0.39000,0.09000,11.27000,54.83000
5,214460270.26900,30.01000,0.72000,1.84000,2.00000,0.46000,13.45000,51.53000
6,190027081.54600,31.19000,0.44000,1.14000,0.97000,0.18000,10.95000,55.13000


In [19]:
cluster_feature = pd.DataFrame([[0, '벼는 익을수록 고개를 숙인다', '높은 신용점수와 대출순위에도 불구하고 적당한 대출에 적은 소비'], 
                                [1, '영끌 소비왕', '소비처가 많아 대출이 필요함에도 낮은 신용점수로 인해 대출이 어려운 3040대'], 
                                [2, '아프니까 청춘이다', '사회 진출 전, 초년생이라는 특성, 저신용으로 인해 대출이 어려운 20대'], 
                                [3, '다들 그렇게 살아간다', '신용도가 중위값에 위치하며 대출이 활발한 그룹 \n다들 그렇게 살아가는거지..'], 
                                [4, '근검절약', '대출도 적고 소비도 적은 고령 그룹\n근검절약!'], 
                                [5, '밑빠진 독에 물 붓기', '대출 받아 써도 써도 모자른 3040 그리고 50대.. 화이팅'], 
                                [6, '인생은 지금부터!', '소비와 대출이 중위, 고신용으로 안정적 신용관리 기대']], 
                                columns=['cluster', 'name', 'explain'])
cluster_feature.set_index('cluster', inplace=True)
cluster_feature = pd.concat([cluster_feature, loan], axis=1)
cluster_feature


,name,explain,total_loan,은행,카드,캐피탈,보험,저축은행,신용,담보
0,벼는 익을수록 고개를 숙인다,높은 신용점수와 대출순위에도 불구하고 적당한 대출에 적은 소비,175156277.53400,34.76000,0.17000,0.35000,0.89000,0.13000,9.37000,54.34000
1,영끌 소비왕,소비처가 많아 대출이 필요함에도 낮은 신용점수로 인해 대출이 어려운 3040대,163625244.12300,27.71000,0.81000,2.90000,1.72000,0.71000,16.86000,49.29000
2,아프니까 청춘이다,"사회 진출 전, 초년생이라는 특성, 저신용으로 인해 대출이 어려운 20대",80698379.62900,23.13000,0.87000,4.74000,1.10000,2.00000,20.27000,47.88000
3,다들 그렇게 살아간다,신용도가 중위값에 위치하며 대출이 활발한 그룹 \n다들 그렇게 살아가는거지..,201372848.66600,29.26000,0.56000,1.96000,1.13000,0.34000,14.66000,52.10000
4,근검절약,대출도 적고 소비도 적은 고령 그룹\n근검절약!,117375599.99900,32.87000,0.12000,0.42000,0.39000,0.09000,11.27000,54.83000
5,밑빠진 독에 물 붓기,대출 받아 써도 써도 모자른 3040 그리고 50대.. 화이팅,214460270.26900,30.01000,0.72000,1.84000,2.00000,0.46000,13.45000,51.53000
6,인생은 지금부터!,"소비와 대출이 중위, 고신용으로 안정적 신용관리 기대",190027081.54600,31.19000,0.44000,1.14000,0.97000,0.18000,10.95000,55.13000


In [20]:
#df.to_csv('./dataset/output_cluster_df.csv', encoding='utf-8', index=False)
#cluster_feature.to_csv('./dataset/output_cluster_feature.csv', encoding='utf-8', index=False)

#### Input & Classification

1. 한달 카드 소비액(체크 + 일시불)
2. 신용점수
3. 도시(인코딩 필요)
4. 연령(인코딩 필요)

In [21]:
def my_info():
    from xgboost import XGBClassifier
    import numpy as np
    import pandas as pd
    
    city_encoding = pd.read_csv('./dataset/city_encoding.csv', encoding='utf-8')
    ages_encoding = pd.read_csv('./dataset/ages_encoding.csv', encoding='utf-8')

    xgb_model = XGBClassifier()
    xgb_model.load_model('xgb_model.model')

    card = int(input('한달 카드 소비 금액 :'))
    score = int(input('신용 점수 :'))
    my_city = input('거주 지역 :')
    my_age = input('나이 :')

    my_values = []
    my_values.append(card)
    my_values.append(score)
    my_values.extend(city_encoding[city_encoding['city']==my_city].values.reshape(-1)[:-1])
    my_values.extend(ages_encoding[ages_encoding['ages'].str.startswith(my_age[0])].values.reshape(-1)[:-1])

    pred = xgb_model.predict([my_values])
    pred_cluster = pred[0]

    return pred_cluster, score


In [22]:
#cluster, score = my_info()

#### Output Class

- 클러스터별 특징
- 자기가 속해 있는 클러스터의 중위값 금액(이상치가 좀 있었기 때문에 이상치의 영향을 덜 받기 위해)- 전체 대출상품
- 비슷한 신용점수를 가진 사람들이 총대출금액을 얼마나 받았는지-위아래 구간의 평균값을 보여주기 +,- 3점
- 대출상품 별 금액비중 순위 (각 클러스터 간 평균대출금액의 합)
- 전체 대출 일시/분할 상환 비중 (inst_rep_loanb(분할상환) | ls_rep_loanb(일시상환)) 의 평균값
- 자신이 그룹 내 신용점수가 상위 몇 프로(분포로)

In [23]:
class output:
    import pandas as pd
    import plotly.express as px
    import plotly.figure_factory as ff

    def __init__(self, cluster_num, credit_score):
        df = pd.read_csv('./dataset/output_cluster_df.csv', encoding='utf-8')
        
        self.cluster_feature = pd.read_csv('./dataset/output_cluster_feature.csv', encoding='utf-8')
        self.my_cluster = cluster_num
        self.my_score = credit_score
        self.dataframe = df[df['cluster']==self.my_cluster]
    
    # 클러스터 이름, 특징
    def feature(self):
        print(self.cluster_feature.iloc[self.my_cluster]['name'])
        print(self.cluster_feature.iloc[self.my_cluster]['explain'])

    # 클러스터 대출 관련 변수 중위값
    def loan_median(self):
        return self.dataframe[['monthly_loan', 'monthly_bk_loan', 'monthly_cd_loan', 'monthly_installments_loan', 'monthly_insurance_loan', 'monthly_sbk_loan', 'credit_loan', 'mortgage_loan']].median()

    # 자신과 비슷한 신용 점수를 가진 사람들의 총 대출 금액 평균
    def total_loan(self):
        avg = self.dataframe[(self.my_score-3 <= self.dataframe['avg_score']) & (self.dataframe['avg_score'] <= self.my_score+3)]['monthly_loan'].mean()

        if np.isnan(avg):
            min = self.dataframe['avg_score'].min()
            max = self.dataframe['avg_score'].max()
            if self.my_score < min:
                return self.dataframe[self.dataframe['avg_score'] == min]['monthly_loan'].mean()
            elif self.my_score > max:
                return self.dataframe[self.dataframe['avg_score'] == max]['monthly_loan'].mean()
        
        return avg


    # 대출 상품 별 금액 비중 순위 (대출 상품 별 평균 대출 금액의 합)
    def loan_rank(self):
        loan = self.cluster_feature.iloc[self.my_cluster][3:].sort_values(ascending=False)
        for i in range(7):
            print('{0}위  {1} ({2}%)'.format(i+1, loan.index[i], round(loan[i],2)))

    # 상환 비중 Pie Chart
    def rep_loanb_pie(self):
        return px.pie(values=[self.dataframe['inst_rep_loanb'].mean(), self.dataframe['ls_rep_loanb'].mean()], names=['inst_rep_loanb', 'ls_rep_loanb'], title='대출 상환 비중')

    # 클러스터 내 신용 점수 분포
    def credit_score_dist(self):
        c_score = self.dataframe['avg_score'].values
        c_score = [c_score]
        fig = ff.create_distplot(c_score, ['신용점수'])
        fig.update_layout(title = '신용점수 분포')
        annotations = []
        annotations.append(dict(x=self.my_score, y=0, text='나의 신용점수', ax=0, ay=-150, borderpad=5, arrowcolor='crimson', arrowwidth=2))
        fig.update_layout(annotations=annotations)
        return fig
    

#### test

In [93]:
info = my_info()
test = output(info[0], info[1])

ValueError: invalid literal for int() with base 10: ''

In [87]:
test.feature()

인생은 지금부터!
소비와 대출이 중위, 고신용으로 안정적 신용관리 기대


In [88]:
test.loan_median()

monthly_loan                 75245000.00000
monthly_bk_loan              56100000.00000
monthly_cd_loan                800000.00000
monthly_installments_loan     2215000.00000
monthly_insurance_loan        1630000.00000
monthly_sbk_loan               320000.00000
credit_loan                  20575000.00000
mortgage_loan               101075000.00000
dtype: float64

In [89]:
test.total_loan()

101550000.0

In [90]:
test.loan_rank()

1위  담보 (55.13%)
2위  은행 (31.19%)
3위  신용 (10.95%)
4위  캐피탈 (1.14%)
5위  보험 (0.97%)
6위  카드 (0.44%)
7위  저축은행 (0.18%)


In [91]:
test.rep_loanb_pie()

In [92]:
test.credit_score_dist()